<a href="https://colab.research.google.com/github/VGoma23/data-520-asean/blob/main/Decision_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import ee
import geemap
ee.Authenticate()
ee.Initialize(project='iuu-fishing-detections-asean')

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
path = '/content/drive/MyDrive/sample_skylight_dataset/978649_1780574_158735'
!ls /content/drive/MyDrive/sample_skylight_dataset

1186117_1894101_158904	464896_2199552_134025  562176_1317888_74483
304128_2197504_133255	542720_1308672_129570  562176_1317888_76285
3875840_1474560_130468	561152_1314816_132052  978649_1780574_158735


In [5]:
import json
data = open(path + '/items.json')
item = json.load(data)
item

[{'layer_name': 'sentinel2',
  'serialized_item_groups': [[{'name': 'S2B_MSIL1C_20230607T164849_N0509_R026_T15RTJ_20230607T203047',
     'geometry': {'projection': {'crs': 'EPSG:4326',
       'x_resolution': 1,
       'y_resolution': 1},
      'shp': 'POLYGON ((-96.00183 26.188591003598358, -94.90387 26.207323832821213, -94.88815 25.216351118358443, -95.97708 25.198425816101338, -96.00183 26.188591003598358))',
      'time_range': ['2023-06-07T16:48:49.024000+00:00',
       '2023-06-07T16:48:49.024000+00:00']},
     'blob_prefix': 'tiles/15/R/TJ/S2B_MSIL1C_20230607T164849_N0509_R026_T15RTJ_20230607T203047.SAFE/GRANULE/L1C_T15RTJ_A032656_20230607T170137/IMG_DATA/T15RTJ_20230607T164849_',
     'cloud_cover': 0.0370270835199618}]],
  'materialized': False}]

In [102]:
bounds_string = item[0]['serialized_item_groups'][0][0]['geometry']['shp'][10:-2]
time_string = item[0]['serialized_item_groups'][0][0]['geometry']['time_range']
print(bounds_string)
print(time_string)

153.13020281543754 47.7440902950869, 153.10589007975994 47.68991071954792, 153.04082621054684 47.54493434441257, 152.97616663111236 47.3998988788116, 152.91208996150098 47.25490447308688, 152.84837828616972 47.10986420932521, 152.78496659158196 46.96479546562996, 152.74081019895922 46.86362481884739, 151.687588500476 46.858175957972115, 151.66286529044567 47.84591482730969, 153.13047036191682 47.85362771191058, 153.13020281543754 47.7440902950869
['2021-11-14T00:59:11.024000+00:00', '2021-11-14T00:59:11.024000+00:00']


In [103]:
## Script to get start time and bounds for the each sample
time_start = time_string[0][0:26]

print(time_start)
bounds_vertices= []
bounds_pairs = [x for x in bounds_string.split(", ")]
for pair in bounds_pairs:
  pair_list = [float(x) for x in pair.split(' ')]
  bounds_vertices.append(pair_list)

print(bounds_vertices)
bounds_polygon = ee.Geometry.Polygon([bounds_vertices])


2021-11-14T00:59:11.024000
[[153.13020281543754, 47.7440902950869], [153.10589007975994, 47.68991071954792], [153.04082621054684, 47.54493434441257], [152.97616663111236, 47.3998988788116], [152.91208996150098, 47.25490447308688], [152.84837828616972, 47.10986420932521], [152.78496659158196, 46.96479546562996], [152.74081019895922, 46.86362481884739], [151.687588500476, 46.858175957972115], [151.66286529044567, 47.84591482730969], [153.13047036191682, 47.85362771191058], [153.13020281543754, 47.7440902950869]]


In [119]:
# Getting each sample from the
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)



image = (
    ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
    .filterDate(time_start, '2025-01-01')
    .filterBounds(bounds_polygon)
    # Pre-filter to get less cloudy granules.
    .map(mask_s2_clouds)
    .first()
)


image

In [108]:
rgb_vis = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

m = geemap.Map()
m.set_center(148.9279137, 47.7440902950869, 12)
m.add_layer(image, rgb_vis, 'RGB')
m

Map(center=[47.7440902950869, 153.13020281543754], controls=(WidgetControl(options=['position', 'transparent_b…

In [6]:

data_2 = open(path + '/layers/label/data.geojson')
labels = json.load(data_2)
labels

{'type': 'FeatureCollection',
 'features': [],
 'properties': {'crs': 'EPSG:32615', 'x_resolution': 10, 'y_resolution': -10}}

In [124]:
metadata_file = open(path + '/layers/sentinel2/R_G_B/metadata.json')
metadata = json.load(metadata_file)
metadata_bounds = metadata["bounds"]
metadata_bounds

[47459, -520968, 48130, -520299]

In [126]:
meta_geom = ee.Geometry.BBox(metadata_bounds[0], metadata_bounds[1], metadata_bounds[2], metadata_bounds[3])

EEException: Geometry.BBox: north must be at least -90°, but was -520299°

What I've Learned


Current Challenges

Dataset format: The labaled dataset has each sample divided into many different files. The satellite images are stored with different bands in different .png files with metadata in a .json file. Most geographic images are stored as a geotiff which holds the metadata in the rasterized image so that it can be placed on a map. because this data is divided I am not sure how to treat the png as a geographic object in a systematic way.
I have tried to

The labels too are stored in another file yet again. Each sample has a label.geojson file that stores the detections formatted as a feature list. The feature list. Feeature Lists are how Google Earth Engine stores data for each satellite images. I do not know how to append

routes forward:
use location and time metadata to pull new images from

Image classification vs object detection: